In [1]:
!wget "https://he-public-data.s3.ap-southeast-1.amazonaws.com/shell_dataset.zip"

--2021-11-07 03:27:14--  https://he-public-data.s3.ap-southeast-1.amazonaws.com/shell_dataset.zip
Resolving he-public-data.s3.ap-southeast-1.amazonaws.com (he-public-data.s3.ap-southeast-1.amazonaws.com)... 52.219.132.187
Connecting to he-public-data.s3.ap-southeast-1.amazonaws.com (he-public-data.s3.ap-southeast-1.amazonaws.com)|52.219.132.187|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3467218047 (3.2G) [binary/octet-stream]
Saving to: ‘shell_dataset.zip’

shell_dataset.zip   100%[===================>]   3.23G  17.3MB/s    in 3m 18s  

2021-11-07 03:30:33 (16.7 MB/s) - ‘shell_dataset.zip’ saved [3467218047/3467218047]



In [2]:
!unzip -q "shell_dataset.zip"
!unzip -q "dataset/train.zip"
!unzip -q "dataset/test.zip"

In [255]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from statsmodels.tsa.statespace.varmax import VARMAX

In [169]:
tdf = pd.read_csv('dataset/test.csv')

In [170]:
tdf = tdf.set_index('scenario_set')

In [171]:
path = 'test/'+str(232)+'/weather_data.csv'
df = pd.read_csv(path)
df.columns

Index(['Time [Mins]', 'Global CMP22 (vent/cor) [W/m^2]', 'Direct sNIP [W/m^2]',
       'Azimuth Angle [degrees]', 'Tower Dry Bulb Temp [deg C]',
       'Tower Wet Bulb Temp [deg C]', 'Tower Dew Point Temp [deg C]',
       'Tower RH [%]', 'Total Cloud Cover [%]', 'Peak Wind Speed @ 6ft [m/s]',
       'Avg Wind Direction @ 6ft [deg from N]', 'Station Pressure [mBar]',
       'Precipitation (Accumulated) [mm]', 'Snow Depth [cm]', 'Moisture',
       'Albedo (CMP11)'],
      dtype='object')

In [172]:
df.corr()['Total Cloud Cover [%]']

Time [Mins]                             -0.341169
Global CMP22 (vent/cor) [W/m^2]          0.167205
Direct sNIP [W/m^2]                     -0.182232
Azimuth Angle [degrees]                 -0.384575
Tower Dry Bulb Temp [deg C]              0.271967
Tower Wet Bulb Temp [deg C]             -0.155039
Tower Dew Point Temp [deg C]            -0.320068
Tower RH [%]                            -0.302800
Total Cloud Cover [%]                    1.000000
Peak Wind Speed @ 6ft [m/s]              0.056552
Avg Wind Direction @ 6ft [deg from N]   -0.067952
Station Pressure [mBar]                 -0.209176
Precipitation (Accumulated) [mm]        -0.290730
Snow Depth [cm]                         -0.100890
Moisture                                -0.308059
Albedo (CMP11)                           0.364291
Name: Total Cloud Cover [%], dtype: float64

In [266]:
for i in range(1,301):
  path = 'test/'+str(i)+'/weather_data.csv'
  df = pd.read_csv(path)
  todays_date = datetime.now().date()
  index = pd.date_range('2020-01-01', periods=361, freq='1min')
  df = df.set_index(index[0:361])
  df['Total Cloud Cover [%]'] = [np.NaN if (per < 0 or per > 100) else per for per in df['Total Cloud Cover [%]']]
  df['Total Cloud Cover [%]'] = df['Total Cloud Cover [%]'].interpolate()
  df['Total Cloud Cover [%]'].fillna(df['Total Cloud Cover [%]'].mean(),inplace = True)
  df = df[['Time [Mins]','Direct sNIP [W/m^2]', 'Total Cloud Cover [%]']]
  model = VARMAX(df, order=(1,0))
  history = model.fit()
  forecast = history.forecast(steps=121)  
  forecast['Total Cloud Cover [%]'] = [0 if (per < 0) else per for per in forecast['Total Cloud Cover [%]']]
  forecast['Total Cloud Cover [%]'] = [100 if (per > 100) else per for per in forecast['Total Cloud Cover [%]']]
  tdf.loc[i,'30_min_horizon'] = forecast.loc['2020-01-01 06:30:00','Total Cloud Cover [%]']
  tdf.loc[i,'60_min_horizon'] = forecast.loc['2020-01-01 07:00:00','Total Cloud Cover [%]']
  tdf.loc[i,'90_min_horizon'] = forecast.loc['2020-01-01 07:30:00','Total Cloud Cover [%]']
  tdf.loc[i,'120_min_horizon'] = forecast.loc['2020-01-01 08:00:00','Total Cloud Cover [%]']

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/statespace/varmax.py:357: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
 

In [267]:
tdf

,30_min_horizon,60_min_horizon,90_min_horizon,120_min_horizon
scenario_set,,,,
1,100.000000,100.000000,100.000000,100.000000
2,78.675104,99.518371,100.000000,100.000000
3,98.926621,98.929993,98.929908,98.929809
4,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...
296,21.809086,23.437758,24.084405,24.319210
297,0.000000,0.000000,0.000000,0.000000
298,0.000000,0.000000,0.000000,0.000000


In [268]:
tdf.to_csv('sol.csv')